In [1]:
#Importiamo spacy con il modello inglese
import spacy
nlp = spacy.load("it_core_news_sm")
text = ('I giovani non hanno bisogno di sermoni, i giovani hanno bisogno di esempi di onestà, di coerenza e di altruismo.')
doc = nlp(text)

# Statistica

In [2]:
#Frequenza Parole
from collections import Counter
words = [token.text for token in doc if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
common_words = word_freq.most_common()
print (common_words)

[('giovani', 2), ('bisogno', 2), ('I', 1), ('sermoni', 1), ('i', 1), ('esempi', 1), ('onestà', 1), ('coerenza', 1), ('e', 1), ('altruismo', 1)]


In [3]:
#Parole Uniche
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

['I', 'sermoni', 'i', 'esempi', 'onestà', 'coerenza', 'e', 'altruismo']


# Speech Tagging

In [4]:
#Dal risultato possiamo vedere come ad ogni parola sia stata associato il "valore grammaticale"
for token in doc:
    print (token, token.pos_)

I DET
giovani NOUN
non ADV
hanno VERB
bisogno NOUN
di ADP
sermoni NOUN
, PUNCT
i DET
giovani NOUN
hanno VERB
bisogno NOUN
di ADP
esempi NOUN
di ADP
onestà NOUN
, PUNCT
di ADP
coerenza NOUN
e CONJ
di ADP
altruismo NOUN
. PUNCT


In [ ]:
#Visualizzazione del tagging grafica
from spacy import displacy
displacy.serve(doc, style='dep')

C:\Users\Marco\Miniconda3\lib\runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



# Lemmatizzazione

In [ ]:
#Processo di tokenizzazione, ossia divisione di una frase in token
for token in doc:
    print (token, token.lemma_)


In [ ]:
#Dal risultato superiori possiamo vedere che le parole "complesse", dove possibile possono essere semplificate.
#ESEMPIO:
# esempi -> esempio
# hanno -> avere

# Sentiment Analysis

In [ ]:
import pandas as pd

#Carichiamo il nostro df
#Questo df contiene una lista di recensioni provenienti da diversi siti come (Amazon, IMDB e Yelp)
df = pd.read_csv(r"C:\Users\Marco\Downloads\sentiment labelled sentences\sentiment labelled sentences\df.csv", encoding='UTF-8')
df.head()

In [ ]:
#Importiamo una lista di stopwords(Parole poco significative per il nostro scopo)
nlp = spacy.load('en_core_web_lg')
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)


In [ ]:
#Esempio di stopwords
stopwords[:10]

In [ ]:
# Importiamo il modulo string per avere la lista di tutta la punteggiatura (./,/;...)
import string
punctuations = string.punctuation

In [ ]:
# Creiamo un "Parser" con spacy impostando lingua inglese
from spacy.lang.en import English
parser = English()


In [ ]:
#A questo punto dovremo "pulire" la nostra frase da tutto ciò che non è necessario a fare il training del nostro algoritmo di ML
#Creiamo quindi una funziona che faccia la "pulizia"
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    #A questo punto ogni "token" ossia ogni parola, che non sia un pronome, viene lemmatizzata ossia portata alla forma base (Es. words -> word),
    #viene portata in lower case e ad essa vengono tolti eventuali spazi vuoti prima e dopo di essa
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    #In seguito vengono tolte tutte le stopwords e la punteggiatura
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

In [ ]:
print(spacy_tokenizer("The commercials are the most misleading.I Hate him"))

In [ ]:
# ML Packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [ ]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [ ]:
# Vettorializzazione
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()

In [ ]:
# Features and Labels
X = df['Recensione']
ylabels = df['Esito']

In [ ]:
# Splittiamo il dataset in test e train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)


In [ ]:
#Creaiamo la pipeline, ossia l'insieme delle azioni che dovranno essere fatte per ottenere i dati elaborati
#Cleaner - Tokenizzazione e pulizia dei dati
#Vettorizzazione dei dati
#Classificazione dei dati
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [ ]:
# Facciamo il fitting dei dati
pipe.fit(X_train,y_train)

In [ ]:
#Proviamo a predirre il dataset di test
sample_prediction = pipe.predict(X_test)

In [ ]:
# Stampiamo i risultati
# 1 = Recensione Positivia
# 0 = Recensione Negativa
for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)

In [ ]:
# Precisione
print("Accuracy: ",pipe.score(X_test,y_test))
print("Accuracy: ",pipe.score(X_test,sample_prediction))

In [ ]:
print("Accuracy: ",pipe.score(X_train,y_train))

In [ ]:
# Another random review
pipe.predict(["My bunny loves these. She knocks over the towers I build, pushes them around and tosses them. They are incredibly durable, so other rabbit owners out there, you can rest assured your bun won't bite through them. It's a toy I trust her with unsupervised. I don't have any good pictures of her playing with them, but here's one of them scattered around her after an exhausting play-session.
"])